_First code cell from custom template `dsml.ipynb` in [user_path]\Anaconda3\envs\dsml\Lib\site-packages\jupyterlab_templates\templates\jupyterlab_templates_. <br>
_See Tim Paine's [`jupyter_lab templates` extention](https://github.com/timkpaine/jupyterlab_templates)._

In [1]:
import sys
from pathlib import Path, PurePath as PPath

print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = str(PPath(this_path).parent)
    else:
        newp = str(PPath(this_path)) 
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp

# Filtered dir() for method discovery:
def filter_module_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work together
              to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]

def get_mdl_pkgs(alib):
    import inspect
    "Inspect module hierarchy on two levels ony."
    for name, mdl in inspect.getmembers(alib, inspect.ismodule):
        print('\n{:>13} : {}'.format(mdl.__name__, filter_dir(mdl)))
        for mdl_name, mdl_sub in inspect.getmembers(mdl, inspect.ismodule):
            if mdl_sub.__doc__:
                print('\n{:>20} : {}'.format(mdl_name, mdl_sub.__doc__.strip()))
                

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import HTML, Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    return HTML('<div style="{}">{}</div>'.format(style, title))


def add_div(div_class, div_start, div_text, output_string=True):
    from IPython import get_ipython
    from IPython.display import HTML, Markdown
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output string, but the cell mode is still in 'code' not 'markdown':
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    The only thing to do is change the cell mode to Markdown.
    If `output_string=False`, the HTML output is displayed in an output cell.
    """
    accepted = ['info', 'warning', 'danger']
    if div_class not in accepted:
        return HTML(f"""<div class="alert"><b>Wrong class:</b> `div_start` is one of {accepted}.
                    </div>""")
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div) #HTML(div)

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()

if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

Python ver: 3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]
Python env: dsml
Currrent dir: C:\Users\catch\Documents\GitHub\AI\Exercises\1_Constraint Satisfaction

Path added to sys.path: C:\Users\catch\Documents\GitHub\AI\Exercises\1_Constraint Satisfaction


<div class="alert alert-info" ><h1>Work In Progress</h1><h3>But addition is DONE!</h3></div>

---
---
# A class for solving crypto arithmetic (CA) puzzles using Z3.

This project emanates from the CA exercise in `AIND-Constraint_Satisfaction.ipynb` and the CA challenges:
>### Cryptarithmetic Challenges
>0. Search online for [more cryptarithmetic puzzles](https://www.reddit.com/r/dailyprogrammer/comments/7p5p2o/20180108_challenge_346_easy_cryptarithmetic_solver/) (or create your own). Come to office hours or join a discussion channel to chat with your peers about the trade-offs between monolithic constraints & splitting up the constraints. (Is one way or another easier to generalize or scale with new problems? Is one of them faster for large or small problems?)
>0. Can you extend the solution to handle complex puzzles (e.g., using multiplication WORD1 x WORD2 = OUTPUT)?
---

# [Reddit] Challenge #346 [Easy] Cryptarithmetic Solver 

## Challenge Input:    
```
"WHAT + WAS + THY == CAUSE"

"HIS + HORSE + IS == SLAIN"

"HERE + SHE == COMES"

"FOR + LACK + OF == TREAD"

"I + WILL + PAY + THE == THEFT"
```

## Bonus:
```
"TEN + HERONS + REST + NEAR + NORTH + SEA + SHORE + AS + TAN + TERNS + SOAR + TO + ENTER + THERE + AS + HERONS + NEST + ON + STONES + AT + SHORE + THREE + STARS + ARE + SEEN + TERN + SNORES + ARE + NEAR == SEVVOTH"

"SO + MANY + MORE + MEN + SEEM + TO + SAY + THAT + THEY + MAY + SOON + TRY + TO + STAY + AT + HOME +  SO + AS + TO + SEE + OR + HEAR + THE + SAME + ONE + MAN + TRY + TO + MEET + THE + TEAM + ON + THE + MOON + AS + HE + HAS + AT + THE + OTHER + TEN == TESTS"

"THIS + A + FIRE + THEREFORE + FOR + ALL + HISTORIES + I + TELL + A + TALE + THAT + FALSIFIES + ITS + TITLE + TIS + A + LIE + THE + TALE + OF + THE + LAST + FIRE + HORSES + LATE + AFTER + THE + FIRST + FATHERS + FORESEE + THE + HORRORS + THE + LAST + FREE + TROLL + TERRIFIES + THE + HORSES + OF + FIRE + THE + TROLL + RESTS + AT + THE + HOLE + OF + LOSSES + IT + IS + THERE + THAT + SHE + STORES + ROLES + OF + LEATHERS + AFTER + SHE + SATISFIES + HER + HATE + OFF + THOSE + FEARS + A + TASTE + RISES + AS + SHE + HEARS + THE + LEAST + FAR + HORSE + THOSE + FAST + HORSES + THAT + FIRST + HEAR + THE + TROLL + FLEE + OFF + TO + THE + FOREST + THE + HORSES + THAT + ALERTS + RAISE + THE + STARES + OF + THE + OTHERS + AS + THE + TROLL + ASSAILS + AT + THE + TOTAL + SHIFT + HER + TEETH + TEAR + HOOF + OFF + TORSO + AS + THE + LAST + HORSE + FORFEITS + ITS + LIFE + THE + FIRST + FATHERS + HEAR + OF + THE + HORRORS + THEIR + FEARS + THAT + THE + FIRES + FOR + THEIR + FEASTS + ARREST + AS + THE + FIRST + FATHERS + RESETTLE + THE + LAST + OF + THE + FIRE + HORSES + THE + LAST + TROLL + HARASSES + THE + FOREST + HEART + FREE + AT + LAST + OF + THE + LAST + TROLL + ALL + OFFER + THEIR + FIRE + HEAT + TO + THE + ASSISTERS + FAR + OFF + THE + TROLL + FASTS + ITS + LIFE + SHORTER + AS + STARS + RISE + THE + HORSES + REST + SAFE + AFTER + ALL + SHARE + HOT + FISH + AS + THEIR + AFFILIATES + TAILOR + A + ROOFS + FOR + THEIR + SAFE == FORTRESSES"
```

---
### Notes from a pythonista:  
<strong> I import z3 the pythonic way:</strong> 
```import z3``` 
instead of  
```from z3 import *``` as seen in numerous tutorials.  

<strong> Why?</strong>
<strong>
1.  It is not recommended as this can corrupt the namespace (some functions from different libraries could have the same name).
2.  I usually inspect a module **in order to learn** about its various functions and methods, so I used the root object for that (anlong with my function `filter_module_dir`).

The slight drawback &mdash; apart from from having to type a little more &mdash; is that some variables such as those returned by the `Solver.check()` function (i.e. `sat, unsat, unkown`), are not defined.  
The following lines use the [`CheckSatResult()` docs](https://z3prover.github.io/api/html/classz3py_1_1_check_sat_result.html) to define them:
</strong>

In [2]:
import z3

sat = z3.z3.CheckSatResult(z3.Z3_L_TRUE)
unsat = z3.z3.CheckSatResult(z3.Z3_L_FALSE)
# else: unkown

## What's in Z3?

The cell below shows how I list a module's methods to learn about them:

In [4]:
from collections import defaultdict

z3_mdl = filter_module_dir(z3)

z3_data = defaultdict(list)
for entry in z3_mdl:
    z3_data[entry[0]].append(entry)

z3_data['a']
z3_data['A']
z3_constants = z3_data['Z']
z3_constants[:10]

['addressof', 'alignment', 'append_log', 'args2params']

['ARRAY',
 'AlgebraicNumRef',
 'And',
 'AndThen',
 'ApplyResult',
 'ApplyResultObj',
 'ArgumentError',
 'ArithRef',
 'ArithSortRef',
 'Array',
 'ArrayRef',
 'ArraySort',
 'ArraySortRef',
 'Ast',
 'AstMap',
 'AstMapObj',
 'AstRef',
 'AstVector',
 'AstVectorObj',
 'AtLeast',
 'AtMost']

['Z3Exception',
 'Z3PPObject',
 'Z3_APP_AST',
 'Z3_ARRAY_SORT',
 'Z3_BOOL_SORT',
 'Z3_BV_SORT',
 'Z3_DATATYPE_SORT',
 'Z3_DEBUG',
 'Z3_DEC_REF_ERROR',
 'Z3_EXCEPTION']

---
---
# The CryptoArithmeticZ3Solver class definition

In [31]:
import z3

sat = z3.z3.CheckSatResult(z3.Z3_L_TRUE)
unsat = z3.z3.CheckSatResult(z3.Z3_L_FALSE)


class CryptoArithmeticZ3Solver():
    """
    A class to solve cryptoarithmetic puzzles with the Z3 solver.
    :Input:
    crypto (str): Can be lower case but the operators (+ or ==) must be surrounded by a space.
                  Additionally:
                  * the string must be unicode
                  * non-alpha characters are ignored (stripped).
    :Output:
    A formated display of the input puzzle [and its solution if found].
    
    :Note:
    The solve_puzzle function accepts a timeout value, which defaults to 20,000 milliseconds.
    """
    def __init__(self, crypto, solution=None, op=' + '):
        try:
            crypto.encode(encoding='ascii')
        except UnicodeEncodeError as err:
            i, j = err.start, err.end
            msg = '\n\033[48;5;155;38;5;198;1m UnicodeEncodeError: \033[0m'
            msg += '\033[48;5;254;1mCryptoArithmeticZ3Solver\033[0m\033[48;5;254m only accepts unicode strings, '
            msg += 'an {} character was found at position {}: {} \033[0m\n'
            print(msg.format(err.encoding, i, crypto[i:j]))

            self.isOK = False
            return None
        
        self.isOK = True
        if len(op) == 1:
            op = ' ' + op + ' ' # needed to sep words
            
        msg_format = 'Crypto format: "WORD + IS + THIS == MUCH"\n'
        if op not in crypto:
            raise TypeError(msg_format + f'Missing operator {op}')
        if ' == ' not in crypto:
            raise TypeError(msg_format + f'Missing equal " == "')
            
        self.op = op
        if self.op == ' * ':
            raise NotImplementedError('Multiplication not yet implemented.')
        
        crypto = crypto.upper()
        
        str_check = crypto.replace(op,'').replace(' == ', '').replace(' ','')
        if not str_check.isalpha():
            print('Warning: non-alpha chars (e.g. apostrophes) ignored.')

        self.n_op = crypto.count(op)
        
        # Process word list
        
        # Get list of words to appear on each line (summands if +)
        words = [w.strip() for w in crypto.split(self.op)]
        tail = words.pop(-1)
        words += [w.strip() for w in tail.split('==')]
        
        # Final cleanup before assignment:
        def _rem_nonalpha(word):
            if word.isalpha():
                return word
            alpha = [c for c in word if c.isalpha()]
            out = ''.join(alpha)
            return out
        
        for i, w in enumerate(words):
            words[i] = _rem_nonalpha(w)
        
        self.words = words
        self.result_word = words[-1]
        
        # Set other props with one loop:
        words_len = list()
        first_ltrs = list()
        letters = set()
        maxl = 0
        for w in words:
            wlen = len(w)
            words_len.append(wlen)
            if wlen > maxl:
                maxl = wlen
            first_ltrs.append(w[0])
            for l in w:
                letters.add(l)
        
        self.max_len = maxl
        self.letters = letters
        self.first_letters = first_ltrs
        self.initial_lines = self._get_initial_lines(crypto)
        
        # these last 3 are set by solve_puzzle method:
        self.satis = None
        self.solution = None
        self.solution_lines = None
    
    
    def _get_word_repr(self, word):
        """
        Return a string of letters in word separated with a space.
        Used in _get_initial_lines().
        """
        s = ''
        for l in word:
            s += f' {l}'
        return s
    
    def _get_letters_repr(self):
        """
        Return a tuple of letters separated by a space:
             (unique letters in puzzle, start_letters, other_letters).
        Used in _solver_setup().
        """
        L, F, O = '', '', ''
        for l in self.letters:
            L += f'{l} '
            if l in self.first_letters:
                F += f'{l} '
            else:
                O += f'{l} '
        return (L[:-1], F[:-1], O[:-1])
    
        
    def _get_initial_lines(self, crypto):
        """Pretty print the cryptarithmetric sting and its solution.
        Arguments:
        ----------
        (str) crypto format: "WORD + IS + THIS == MUCH"
        (dict) solution: letter as key
        """
        # for aligned printing
        w1 = self.max_len*2
        w2 = 4 + self.max_len*2
            
        lines = [] # -> [('T W O ', 'TWO')]
        line = ''
        
        for p in range(self.n_op + 2):    
            if p == (self.n_op + 1):
                sub = '{0:{fill}>{width}}'.format('-', fill='-', width=4+w1)
                lines.append((sub, '-'))
            
            s = self._get_word_repr(self.words[p])
            
            if (p > 0) and (p < (self.n_op + 1)):
                line = '{op} {0:{fill}>{width}}'.format(s,
                                                  op=self.op,
                                                  fill=' ',
                                                  width=w1)
            else:
                line = '{:{fill}>{width}}'.format(s, fill=' ', width=w2)
            lines.append((line, self.words[p]))
            
        return lines
    
    
    def _get_solution_lines(self):
        if self.solution is None:
            print('Func: _get_solution_lines :: self.solution (sol) is None')
            return []
        else:
            m = self.solution    # :: solver model
            out = dict((str(v), m[v]) for v in m.decls())
            
            # for aligned printing
            w1 = self.max_len*2 - 2
            w2 = self.max_len*2 + 2
            
            lines = [] 
            # get the initial line & expand them with solution
            # (line, word) = [('T W O ', 'TWO')]
            for i, (line, word) in enumerate(self.initial_lines):
                values = ''
                if i == (self.n_op + 1):
                    line += f'    {line}'
                else:
                    for l in word:
                        values += f'{out[str(l)]}'
                    
                    if (i > 0) and (i < len(self.initial_lines)-1):
                        line += '  -> ' + '{op} {0:{fill}>{width}}'.format(values,
                                                                           op=self.op,
                                                                           fill=' ',
                                                                           width=w1)
                    else:
                        line += '  -> ' + '{0:{fill}>{width}}'.format(values,
                                                                      fill=' ',
                                                                      width=w2)
                # match lines format of _get_initial_lines for displaying
                lines.append((line, None))
    
            return lines
        
        
    def display(self):
        if self.solution is None:
            lines_out = self.initial_lines
        else:
            # display only if sat:
            if self.satis == sat:
                lines_out = self.solution_lines
            else:
                return
        for L in lines_out:
            print(f'{L[0]}')
            
        
    def _solver_setup(self):
        # create a Z3 CSP solver instance
        solver = z3.Solver()
        
        # get 'l e t t e r s':
        ltrs, first_ltrs, other_ltrs = self._get_letters_repr()

        # create variables instance for all unique letters:
        ltrs_vars = z3.Ints(ltrs)
        # constraint: all must be different
        c_distinct = [ z3.Distinct(ltrs_vars) ]
        
        # constraint: each word's 1st letter is a non-zero digit:
        c_zero_not_first = [ z3.And(1<=L, L<=9) for L in z3.Ints(first_ltrs) ]
    
        # the others can be 0
        c_digit = [ z3.And(0<=L, L<=9) for L in z3.Ints(other_ltrs) ]

        # add all input constraints to the solver:
        c_input = c_distinct + c_zero_not_first + c_digit
        solver.add(c_input)
        
        return solver

    def solve_puzzle(self, t_out=20000):
        solver = self._solver_setup()
        
        # 2 min timeout
        solver.set(timeout=t_out) 
        ok = solver.check()
            
        if ok == sat:
            self.satis = sat
            self.solution = solver.model()
            self.solution_lines = self._get_solution_lines()
        else:
            print(f'Failed to solve: {str(ok).upper()} solution.\n')
            self.solution = None

---
# Tests!

In [7]:
test_crypto_short = ["WHAT + WAS + THY == CAUSE",
                     "HIS + HORSE + IS == SLAIN",
                     "HERE + SHE == COMES",
                     "FOR + LACK + OF == TREAD",
                     "I + WILL + PAY + THE == THEFT"]

test_crypto_long = ["TEN + HERONS + REST + NEAR + NORTH + SEA + SHORE + AS + TAN + TERNS + SOAR + TO + ENTER + THERE + AS + HERONS + NEST + ON + STONES + AT + SHORE + THREE + STARS + ARE + SEEN + TERN + SNORES + ARE + NEAR == SEVVOTH",
                    "SO + MANY + MORE + MEN + SEEM + TO + SAY + THAT + THEY + MAY + SOON + TRY + TO + STAY + AT + HOME +  SO + AS + TO + SEE + OR + HEAR + THE + SAME + ONE + MAN + TRY + TO + MEET + THE + TEAM + ON + THE + MOON + AS + HE + HAS + AT + THE + OTHER + TEN == TESTS"]
test_crypto_super = ["THIS + A + FIRE + THEREFORE + FOR + ALL + HISTORIES + I + TELL + A + TALE + THAT + FALSIFIES + ITS + TITLE + TIS + A + LIE + THE + TALE + OF + THE + LAST + FIRE + HORSES + LATE + AFTER + THE + FIRST + FATHERS + FORESEE + THE + HORRORS + THE + LAST + FREE + TROLL + TERRIFIES + THE + HORSES + OF + FIRE + THE + TROLL + RESTS + AT + THE + HOLE + OF + LOSSES + IT + IS + THERE + THAT + SHE + STORES + ROLES + OF + LEATHERS + AFTER + SHE + SATISFIES + HER + HATE + OFF + THOSE + FEARS + A + TASTE + RISES + AS + SHE + HEARS + THE + LEAST + FAR + HORSE + THOSE + FAST + HORSES + THAT + FIRST + HEAR + THE + TROLL + FLEE + OFF + TO + THE + FOREST + THE + HORSES + THAT + ALERTS + RAISE + THE + STARES + OF + THE + OTHERS + AS + THE + TROLL + ASSAILS + AT + THE + TOTAL + SHIFT + HER + TEETH + TEAR + HOOF + OFF + TORSO + AS + THE + LAST + HORSE + FORFEITS + ITS + LIFE + THE + FIRST + FATHERS + HEAR + OF + THE + HORRORS + THEIR + FEARS + THAT + THE + FIRES + FOR + THEIR + FEASTS + ARREST + AS + THE + FIRST + FATHERS + RESETTLE + THE + LAST + OF + THE + FIRE + HORSES + THE + LAST + TROLL + HARASSES + THE + FOREST + HEART + FREE + AT + LAST + OF + THE + LAST + TROLL + ALL + OFFER + THEIR + FIRE + HEAT + TO + THE + ASSISTERS + FAR + OFF + THE + TROLL + FASTS + ITS + LIFE + SHORTER + AS + STARS + RISE + THE + HORSES + REST + SAFE + AFTER + ALL + SHARE + HOT + FISH + AS + THEIR + AFFILIATES + TAILOR + A + ROOFS + FOR + THEIR + SAFE == FORTRESSES"]
test_crypto_unsolvable = ["Les + sanglots + longs + Des + violons + De + l'automne + Blessent + mon + coeur + D'une + langueur == Monotone"]
test_crypto_fail = ["Les + sanglots + longs + Des + violons + De + l'automne + Blessent + mon + cœur + D'une + langueur == Monotone"]

# Test for additive cryptoarithmetic puzzles:

In [32]:
def run_test(tests):
    all_cryptos = {'short': test_crypto_short,
                   'long': test_crypto_long,
                   'super': test_crypto_super,
                   'unsolvable': test_crypto_unsolvable,
                   'fail': test_crypto_fail}

    for k, kind in tests.items():
        if k == 'unsolvable':
            display(Markdown(f'### Solving {k.upper()} puzzle(s). Needs ~ 30+ seconds:<br><br>'))
        else:
            display(Markdown(f'### Solving {k.upper()} puzzle(s):<br><br>'))
        for crypt_str in kind:
            crypto = CryptoArithmeticZ3Solver(crypt_str)
            if not crypto.isOK:
                print('Initialization failed.')
                continue
            else:
                if k == 'super':
                    print(f'Number of words: {len(crypto.words)}')
                crypto.solve_puzzle()
                crypto.display()
                print() 

In [30]:
run_test(all_cryptos)

### Solving SHORT puzzle(s):<br><br>

       W H A T  ->         6758
 +       W A S  ->  +       653
 +       T H Y  ->  +       872
--------------    --------------
     C A U S E  ->        15430

         H I S  ->          438
 +   H O R S E  ->  +     46087
 +         I S  ->  +        38
--------------    --------------
     S L A I N  ->        82531

       H E R E  ->         4515
 +       S H E  ->  +       345
--------------    --------------
     C O M E S  ->        26053

         F O R  ->          175
 +     L A C K  ->  +      3208
 +         O F  ->  +        71
--------------    --------------
     T R E A D  ->        65429

             I  ->            2
 +     W I L L  ->  +      4211
 +       P A Y  ->  +       873
 +       T H E  ->  +       965
--------------    --------------
     T H E F T  ->        96509



### Solving LONG puzzle(s):<br><br>

             T E N  ->              376
 +     H E R O N S  ->  +        572169
 +         R E S T  ->  +          2793
 +         N E A R  ->  +          6782
 +       N O R T H  ->  +         61235
 +           S E A  ->  +           978
 +       S H O R E  ->  +         95127
 +             A S  ->  +            89
 +           T A N  ->  +           386
 +       T E R N S  ->  +         37269
 +         S O A R  ->  +          9182
 +             T O  ->  +            31
 +       E N T E R  ->  +         76372
 +       T H E R E  ->  +         35727
 +             A S  ->  +            89
 +     H E R O N S  ->  +        572169
 +         N E S T  ->  +          6793
 +             O N  ->  +            16
 +     S T O N E S  ->  +        931679
 +             A T  ->  +            83
 +       S H O R E  ->  +         95127
 +       T H R E E  ->  +         35277
 +       S T A R S  ->  +         93829
 +           A R E  ->  +           827
 +         S E E N  ->  +          9776


### Solving SUPER puzzle(s):<br><br>

Number of words: 200
                 T H I S  ->                   6432
 +                     A  ->  +                   1
 +               F I R E  ->  +                8390
 +     T H E R E F O R E  ->  +           640908590
 +                 F O R  ->  +                 859
 +                 A L L  ->  +                 177
 +     H I S T O R I E S  ->  +           432659302
 +                     I  ->  +                   3
 +               T E L L  ->  +                6077
 +                     A  ->  +                   1
 +               T A L E  ->  +                6170
 +               T H A T  ->  +                6416
 +     F A L S I F I E S  ->  +           817238302
 +                 I T S  ->  +                 362
 +             T I T L E  ->  +               63670
 +                 T I S  ->  +                 632
 +                     A  ->  +                   1
 +                 L I E  ->  +                 730
 +                 T H E  ->  +            

### Solving UNSOLVABLE puzzle(s). Needs ~ 30+ seconds:<br><br>

Failed to solve: UNKNOWN solution.

               L E S
 +   S A N G L O T S
 +         L O N G S
 +             D E S
 +     V I O L O N S
 +               D E
 +   L A U T O M N E
 +   B L E S S E N T
 +             M O N
 +         C O E U R
 +           D U N E
 +   L A N G U E U R
--------------------
     M O N O T O N E



### Solving FAIL puzzle(s):<br><br>


 UnicodeEncodeError: CryptoArithmeticZ3Solver only accepts unicode strings, an ascii character was found at position 76: œ 

Initialization failed.


---